In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Check device
print(device)

# Parameters (feel free to modify according to your device's specs
batch_size = 32
block_size = 128
max_iters = 300
learning_rate = 2e-5
eval_iters = 50
n_embd = 300
n_head = 4
n_layer = 4
dropout = 0.2

cpu


In [2]:
# Load vocabulary
chars = ""
with open("vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [3]:
# Encode, decode functions
string_to_int = { ch : i for i, ch in enumerate(chars) }
int_to_string = { i : ch for i, ch in enumerate(chars) }

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
# Memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "train_split.txt" if split == 'train' else "val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size * batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size * batch_size - 1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

In [8]:
class Head(nn.Module):
    """ One head of self-attention that processes queries, keys, and values """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)

        # Create a lower triangular matrix for masked attention
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, time_step, channels = x.shape
        
        # Project input into key, query, value representations
        keys = self.key(x)      # Shape: (batch_size, time_step, head_size)
        queries = self.query(x)  # Shape: (batch_size, time_step, head_size)
        values = self.value(x)   # Shape: (batch_size, time_step, head_size)
        
        # Compute attention scores 
        # (batch_size, time_step, time_step) = (batch_size, time_step, head_size) @ (batch_size, head_size, time_step)
        attention_scores = queries @ keys.transpose(-2,-1) * keys.shape[-1]**-0.5
        
        # Apply causal mask to prevent attending to future tokens
        attention_scores = attention_scores.masked_fill(self.tril[:time_step, :time_step] == 0, float('-inf'))
        
        # Apply softmax to get attention weights
        attention_weights = F.softmax(attention_scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Compute weighted sum of values
        output = attention_weights @ values  # (batch_size, time_step, head_size)
        return output

# [1, 0, 0]
# [1, 0.6, 0]
# [1, 0.6, 0.4]
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention processing in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Concatenate outputs from all heads
        # Each head processes the input differently and produces its own attention pattern
        output = torch.cat([head(x) for head in self.heads], dim=-1)

        # Project concatenated outputs back to embedding dimension
        output = self.dropout(self.projection(output))
        return output

class FeedFoward(nn.Module):
    """ A simple linear layer followed by non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    """ Transformer block: Communication followed by Computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.self_attention = MultiHeadAttention(n_head, head_size)
        self.feed_forward = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.self_attention(x)
        x = self.ln1(x + y)
        y = self.feed_forward(x)
        x = self.ln2(x + y)

        return x
    
class GPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        batch_size, time_step = index.shape
        
        # Create token and position embeddings
        token_embeddings = self.token_embedding_table(index)  # (batch_size, time_step, channels)
        position_embeddings = self.position_embedding_table(torch.arange(time_step, device=device))  # (time_step, channels)
        
        # Combine token and position embeddings
        x = token_embeddings + position_embeddings  # (batch_size, time_step, channels)
        
        # Pass through transformer blocks
        x = self.blocks(x)  # (batch_size, time_step, channels)
        x = self.ln_f(x)    # Final layer normalization
        
        # Project to vocabulary size
        logits = self.lm_head(x)  # (batch_size, time_step, vocab_size)

        if targets is None:
            loss = None
        else:
            batch_size, time_step, channels = logits.shape
            logits = logits.view(batch_size * time_step, channels)
            targets = targets.view(batch_size * time_step)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Index is a (batch_size, time_step) array of indices in the current context
        for _ in range(max_new_tokens):
            
            # Crop index to the last 'block_size' tokens
            index_cond = index[:, -block_size:]
            
            # Get the predictions
            logits, loss = self.forward(index_cond)
            
            # Focus only on the last time step
            logits = logits[:, -1, :] # becomes (batch_size, channels)
            
            # Apply 'softmax' to get probabilities
            probs = F.softmax(logits, dim=-1) # (batch_size, channels)
            
            # Sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (batch_size, 1)
            
            # Append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (batch_size, time_step + 1)
        
        return index

model = GPT(vocab_size)
try:
    with open('model-01.pkl', 'rb') as f:
        print('Loading model parameters...')
        model = pickle.load(f)
        print('Loaded successfully!')
except FileNotFoundError:
    pass
finally:
    m = model.to(device)


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [25]:
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('Model saved')

step: 0, train loss: 2.754, val loss: 2.702
step: 25, train loss: 2.722, val loss: 2.672
step: 50, train loss: 2.708, val loss: 2.671
step: 75, train loss: 2.674, val loss: 2.661
2.695997953414917
Model saved


In [29]:
prompt = 'Hello'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
print(generated_chars)

Hellor je sorat nt mor me s cospeleredëltt iw s nenAs 7 s altNecfitinteroul[atchas tàe4ibchess dhe5eatane


In [ ]:
# More training, better results :)